<a href="https://colab.research.google.com/github/AlyonaSarapina/python-and-cyber/blob/main/Testing_PythonProject_lambda_function.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Testing the lambda function code locally as a part of the PythonProject

# Connecting to AWS Simple Storage System (S3)


### Installing boto3 for use in this worksheet

In [1]:
!pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 4.8 MB/s eta 0:00:00


### Saving my access key, and secret key in an environment variable

In [2]:
import os
from IPython.display import clear_output

def set_environment_variable_values():
  ACCESS_KEY = input("Please enter the AWS access key: ")
  SECRET_ACCESS_KEY = input("Please enter the AWS secret access key: ")
  BUCKET_NAME = input("Please enter the name of the bucket in S3: ")
  os.environ['ACCESS_KEY'] = ACCESS_KEY
  os.environ['SECRET_ACCESS_KEY'] = SECRET_ACCESS_KEY
  os.environ['BUCKET_NAME'] = BUCKET_NAME
  clear_output()
  return None

set_environment_variable_values()


### Creating a connection to the S3 bucket

In [3]:
import boto3

def get_S3_client():
	resource = boto3.client(
     "s3",
		aws_access_key_id = os.environ.get('ACCESS_KEY'),
		aws_secret_access_key = os.environ.get('SECRET_ACCESS_KEY')
	)
	return resource

s3_client = get_S3_client()
print(s3_client)

### LAMBDA FUNCTIONS


In [82]:
import pandas as pd
import io

def add_school_data_to_bucket(filename, filedata):
    try:
      file_object = s3_client.get_object(Bucket=os.environ.get('BUCKET_NAME'), Key=filename)
      data_file = io.BytesIO(file_object['Body'].read())
      data = pd.read_csv(data_file)
      data_list = data.values.tolist()
      names_array = []
      for row in data_list:
        names_array.append(row[0])
      new_data = []
      for school in filedata:
        if school[0] not in names_array:
          new_data.append(school)
      data_list = data_list.append(new_data)
      if len(new_data) == 0:
        return "This schools already in a file", []
      else:
        file_object =  io.StringIO()
        pd.DataFrame(data_list).to_csv(file_object, index=False)
        s3_client.put_object(Bucket=os.environ.get('BUCKET_NAME'), Body=file_object.getvalue(), Key=filename)
      return "New data successfully added", new_data
    except Exception as e:
      if str(e) == "An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.":
        headers = ["Name", 'Latitude', 'Longitude']
        file_object =  io.StringIO()
        pd.DataFrame(filedata, columns=headers).to_csv(file_object, index=False)
        s3_client.put_object(Bucket=os.environ.get('BUCKET_NAME'), Body=file_object.getvalue(), Key=filename)
        return "New file successfully created", filedata
      else:
       return str(e), []

def show_schools_data_in_bucket(filename):
    try:
        file_object = s3_client.get_object(Bucket=os.environ.get('BUCKET_NAME'), Key=filename)

        data_file = io.BytesIO(file_object['Body'].read())
        data = pd.read_csv(data_file)
        data_list = data.values.tolist()
        return "The data has been found", data_list
    except Exception as e:
        return "There is no such file", []




In [70]:
import json

def lambda_handler(event, context):
    global message, return_data, statuscode
    filename = "schools_list.csv"
    if event["httpMethod"] == "POST":
        if "body" in event.keys():
            request = event["body"]
            if type(request) is not dict:
                request = json.loads(request)
            if request is not None and "data" in request.keys():
                data = request["data"]
                if len(data) == 0:
                    message, return_data = "Please enter a valid data", []
                    statuscode = 404
                else:
                    message, return_data = add_school_data_to_bucket(filename, data)
                    statuscode = 200
            else:
                message, return_data = "Error in the POST request occured", []
                statuscode = 404
    elif event["httpMethod"] == "GET":
       message, return_data = show_schools_data_in_bucket(filename)
       statuscode = 200
    else:
        message, return_data = "Error occured", []
        statuscode = 404
    return {'statusCode': statuscode,
            'headers': {'Content-Type': 'application/json',
                        'Access-Control-Allow-Headers': 'Content-Type,X-Api-Key',
                        'Access-Control-Allow-Methods': 'POST',
                        'Access-Control-Allow-Origin': '*'},
            'body': {"message": message, "data": []}
            }


Test 1 - creating a new file

In [90]:
event= {
    "httpMethod": "POST",
    "body": {
    "data": [
        [
            "Bannockburn Primary School",
            51.4869172,
            0.1015561
        ],
        [
            "St Margaret Clitherow Primary School",
            51.50103289999999,
            0.1132992
        ]
    ]
}
}

test1 = lambda_handler(event, None)

if test1['body']['message'] == "New file successfully created":
  print("Test 1 passed")
else:
  print("Test 1 failed")

Test 1 passed


Test 2 - adding a new data in a new file

In [87]:
event = {
    "httpMethod": "POST",
    "body": {
    "data": [
        [
        'Jesmond Park Academy',
         54.9939443,
         -1.588097
        ]
    ]
}
}

test2 = lambda_handler(event, None)

if test2['body']['message'] == "New data successfully added":
  print("Test 2 passed")
else:
  print("Test 2 failed")

Test 2 passed


Test 3 - GET request when file doesn't exists

In [85]:
event= {
    "httpMethod": "GET",
}

test3 = lambda_handler(event, None)

if test3['body']['message'] == "There is no such file":
  print("Test 3 passed")
else:
  print("Test 3 failed")




Test 3 passed


Test 4 - GET request of file with data

In [91]:
event= {
    "httpMethod": "GET",
}

test4 = lambda_handler(event, None)

if test4['body']['message'] == "The data has been found":
  print("Test 4 passed")
else:
  print("Test 4 failed")

Test 4 passed
